In [ ]:
import sklearn
from sklearn.cluster import KMeans,MeanShift,AffinityPropagation
from sklearn.mixture import GaussianMixture
from ConstructDatatset import *
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from TotalCalculation import *
from preprocessing import *
from ClassPrediction import *

In [ ]:
df,filename = Builddataset().concateDataset(folder_name = r'C:\Users\ASUS\Documents\NARIT_internship_data\CSV_dataset_sobel')

In [ ]:
name_y = str('Blue channel')
name_x = np.arange(len(df))
dintr = np.gradient(df['Blue channel'])
dintb = np.gradient(df['Red channel'])

plt.figure(figsize=(30,20))
sns.lineplot(data=df,x=name_x,y=df[name_y],c='blue')
sns.lineplot(data=df,x=name_x,y=df['Red channel'],c='green')
sns.lineplot(data=df,x=name_x,y=df['intensity'])
plt.plot(dintr)
plt.plot(dintb,c='r')

In [ ]:
date = []
for i in df.index.tolist():
    date.append(timeConvertion().ticks_to_datetime(i,time_zone=7))
df['Time'] = date


In [ ]:
df

In [ ]:
df['Time (decimal)'] = df['Time'].dt.hour+df['Time'].dt.minute/60

In [ ]:
start_date = '2024-01-01'
end_date = str(df['Time'].tolist()[1])
location = [18.849417,98.9538]
days = timeConvertion().time_duration(start_date,end_date,include_end_date=True).days

LSTM = SunPosition.LSTM(time_zone_offset=7)
EoT = SunPosition.calculate_EoT(day=days)
TC = SunPosition.TimeCorrectionFactor(Longitude=location[1],LSTM=LSTM,EoT=EoT)
dec = SunPosition.declination(day=days)
suntime = SunPosition.DaytimeInfo(latitude=location[0],declination=dec,TC=TC)
sunrise,sunset = SunPosition.DaytimeInfo(latitude=location[0],declination=dec,TC=TC)

In [ ]:
df = df[(df['Time (decimal)'] > sunrise) & (df['Time (decimal)'] < sunset)]

In [ ]:
x = df.drop(columns=['Time (decimal)','Time','Red channel','Blue channel','intensity'])

dataset = list(x.itertuples(index=False,name=None))
dataset = np.array(dataset)

In [ ]:
df.head(10)

In [ ]:
sns.heatmap(data=x.corr(),cmap='jet',annot=True)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
np.random.seed(42)

kmeans = KMeans(n_clusters=7, tol=1e-5, random_state=42, init='random' ,n_init=20, max_iter=500, algorithm='elkan')
#af = AffinityPropagation(damping=0.95, max_iter=500, convergence_iter=20, preference=-50, affinity='euclidean')
GMM = GaussianMixture(n_components=7, tol=1e-4, init_params='kmeans',covariance_type='full', max_iter=500, random_state=42, n_init=10)
labels = GMM.fit_predict(dataset)
labels_kmean = kmeans.fit_predict(dataset)
#labels_af = af.fit_predict(dataset)

In [ ]:
df.loc[:, 'Label_GMM'] = labels
df.loc[:, 'Label_Kmean'] = labels_kmean
#df.loc[:, 'Label_AF'] = labels_af

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
sns.scatterplot(data=df,x=df['dissimilarity'],y=df['homogeneity'],hue=labels,palette='rainbow')
plt.subplot(1,2,2)
sns.scatterplot(data=df,x=df['dissimilarity'],y=df['homogeneity'],hue=labels_kmean,palette='rainbow')


In [ ]:
eva = Evaluation()
eva.silhouette(model=kmeans,data=dataset,n=11)

import pickle 

with open('af_model_1.pkl','wb') as f:
    pickle.dump(af,f)

import pickle
with open('gmm_model_2.pkl', 'wb') as f:
    pickle.dump(GMM, f)
with open('kmean_model_2.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

from preprocessing import image
image_list = image.getFilename(r'C:\Users\ASUS\Documents\NARIT_internship_data\Test_folder')

filetime = []
for file in image_list:
    filetime.append(int(os.path.splitext(os.path.basename(file))[0]))


viz = visualizer()
number = 1
source_folder = r'All_sky_camera_Astropark_Chaingmai\2024-06\2024-06-12'
destination_folder = r'C:\Users\ASUS\Documents\NARIT_internship_2024\Tester'

viz.copy_matching_files(df=viz.match_label(df=df,number=number), source_folder=source_folder, destination_folder=destination_folder)

pred = prediction()
path = r'All_sky_camera_Astropark_Chaingmai\2024-06\2024-06-09\638534868144584540.png'
pred_1,pred_2,cloud_percentage,sky_status,final = pred.total_prediction(image_path=path,mask_path='mask_delete_5.png',crop_size=570,kmeans=kmeans,GMM=GMM,df=df)

mask = cv2.imread('mask_delete_5.png',cv2.IMREAD_GRAYSCALE)
mask = crop_center(img=mask,crop_size=570)
img,name = load_single_image(path=path,crop_size=570,mask=mask,apply_crop_sun=False)
image,value,RB = pred.RBsingle(input=img,dataframe=df)
cloud_ratio = pred.CloudRatio(image=img,mask=mask)

import requests
import cv2
import numpy as np
import hashlib
import time

def get_image_hash(image_data):
    return hashlib.md5(image_data).hexdigest()

image_url = "http://weather.narit.or.th/images/allskyimage/ASTROPARK/2024-08-19"
previous_hash = None

while True:
    # Fetch the image from the web
    response = requests.get(image_url)
    if response.status_code == 200:
        # Get the current hash of the image
        current_hash = get_image_hash(response.content)
        
        # Compare with the previous hash
        if current_hash != previous_hash:
            # The image has changed
            print("Image has changed, updating display...")
            image_array = np.frombuffer(response.content, np.uint8)
            image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
            cv2.imshow('Auto-Updated Image', image)
            
            # Update the previous hash
            previous_hash = current_hash
            
            # Wait for 1 second to avoid rapid flickering
            if cv2.waitKey(1000) & 0xFF == ord('q'):
                break
        else:
            print("No change detected.")
    
    # Check every 10 seconds for a change
    time.sleep(10)
    cv2.destroyAllWindows()
